N-граммы - это языковые модели, которые используются для предсказания N-ного слова исходя из предыдущих N-1 слов.

P(w<sub>i</sub> | w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>) = C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>, w<sub>i</sub>) / C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>)  (1)

Пример: биграммы (N = 2)

In [ ]:
corpus = "John read her book. I read a different book. John read a book by Mulan."

Чтобы P(w<sub>i</sub> | w<sub>i-1</sub>) имело смысл для i = 1, добавим в начало каждого предложения специальный токен &lt;s&gt;. В общем случае таких токенов должно быть N - 1.

Чтобы вероятности всех последовательностей в сумме составляли 1, добавим в конец каждого предложения специальный токен &lt;/s&gt;

Теперь посчитаем вероятность P(John read a book):

P(John read a book) = P(John | &lt;s&gt;) * P(read | John) * P(a | read) * P(book | a) * P(&lt;/s&gt; | book)

А какова вероятность P(Mulan read a book)?

Сглаживание Лапласа: будем делать вид, что все возможные биграммы встретились на один раз больше, чем в реальности. Для этого в формуле (1) прибавим в числителе 1, а в знаменателе - размер словаря (включая &lt;s&gt; и &lt;/s&gt;):

P(w<sub>i</sub> | w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>) = (C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>, w<sub>i</sub>) + 1) / (C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>) + V) (2)

Обработка неизвестных слов: добавим в словарь токен &lt;unk&gt;, на который будем заменять все незнакомые слова (которые не входят в словарь). Благодаря сглаживанию Лапласа для всех N-грамм с этим словом будут ненулевые вероятности.

Задание: обучить две N-граммные модели на корпусе текстов студенческой конференции (две разные секции). Взять текст, не входящий в обучающую выборку, и попробовать определить его вероятность по каждой из двух моделей. Где вероятность больше, той секции, скорее всего, он и будет принадлежать.